<a href="https://colab.research.google.com/github/SkyChen1009/ML-project/blob/main/Titanic%20-%20ML%20Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Install package
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.1-py2.py3-none-any.whl size=257484150 sha256=246b5e98f86289035b46db354977e55fbecb3c126eb88e8f19499c7e9523d9e2
  Stored in directory: /root/.cache/pip/wheels/d9/9b/ca/7345b72d17e1e17da37239d70631c3214ec9e541b0c9e700e2
Successfully built h2o


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data preprocessing
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# metric
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
testdata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gender_submission.csv')

# EDA

In [ ]:
print('train_shape: ', train.shape)
print('test_shape: ', testdata.shape)
print('sub_shape: ', sub.shape)

train_shape:  (891, 12)
test_shape:  (418, 11)
sub_shape:  (418, 2)


In [ ]:
print(train.info())
print(testdata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

In [ ]:
# check nunique
for column in train.columns:
    unique_count = train[column].nunique()
    print(f"{column}: {unique_count}")

PassengerId: 891
Survived: 2
Pclass: 3
Name: 891
Sex: 2
Age: 88
SibSp: 7
Parch: 7
Ticket: 681
Fare: 248
Cabin: 147
Embarked: 3


In [ ]:
print(train.isnull().sum())
print(testdata.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


# Data Preprocessing

In [ ]:
# Fill missing value
age_mean = train['Age'].mean()
fare_mean = train['Fare'].mean()

train['Age'].fillna(age_mean, inplace=True)
testdata['Age'].fillna(age_mean, inplace=True)

testdata['Fare'].fillna(fare_mean, inplace=True)

In [ ]:
imputer = SimpleImputer(strategy='most_frequent')
missing_columns = ['Cabin', 'Embarked']

for column in missing_columns:
  combined_data = pd.concat([train[column], testdata[column]], axis=0)
  combined_data = pd.DataFrame(combined_data, columns=[column])
  combined_data = imputer.fit_transform(combined_data)

  train[column] = combined_data[:len(train)]
  testdata[column] = combined_data[len(train):]

In [ ]:
train['Sex'] = train['Sex'].replace({'male': 1, 'female': 0})
testdata['Sex'] = testdata['Sex'].replace({'male': 1, 'female': 0})

In [ ]:
label_encoder = LabelEncoder()
object_columns = ['Name', 'Ticket', 'Cabin', 'Embarked']

for column in object_columns:
    combined_data = pd.concat([train[column], testdata[column]], axis=0)
    label_encoder.fit(combined_data)

    train[column] = label_encoder.transform(train[column])
    testdata[column] = label_encoder.transform(testdata[column])

In [ ]:
# Finding correlation of every variables with target
correlation_matrix = train.corr()
correlation_with_target = correlation_matrix['Survived']
print(correlation_with_target)

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Name          -0.057487
Sex           -0.543351
Age           -0.069809
SibSp         -0.035322
Parch          0.081629
Ticket        -0.166734
Fare           0.257307
Cabin          0.103213
Embarked      -0.167675
Name: Survived, dtype: float64


In [ ]:
testdata.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
# Drop unimportant features
features_to_drop = ['PassengerId', 'Name']

train = train.drop(features_to_drop, axis=1)
testdata = testdata.drop(features_to_drop, axis=1).copy()

In [ ]:
# train test split
X = train.drop(['Survived'], axis=1)
y = train['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(fit_intercept=True, max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8100558659217877


# Ensemble Learning: Stacking

In [ ]:
# Stacking
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Cross validation strategy
cv = KFold(n_splits=10, random_state=42, shuffle=True)

# Define meta model
meta_model = LogisticRegression(fit_intercept=True, max_iter=1000)

# Define base model
base_model = base_models = [
    ('rf_model', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm_model', SVC()),
    ('xgb_model', XGBClassifier(objective='binary:logistic', learning_rate=0.5, max_depth=3))
    # Add more base models if needed
]

# Stacking
model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=cv)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Performance
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8212290502793296

## Test and Submission

In [ ]:
pred = model.predict(testdata)
sub['Survived'] = pred
sub.to_csv('/content/drive/MyDrive/Colab Notebooks/sub.csv', index=False, header=True)

# H2O AutoML

In [ ]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 0 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_2kd6d9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.092 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
trainframe = h2o.H2OFrame(train)
testframe = h2o.H2OFrame(testdata)
trainf, testf, validf = trainframe.split_frame([0.6, 0.2], seed=42)
trainf['Survived'] = trainf['Survived'].asfactor()
y = 'Survived'
X = list(trainframe.columns)
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
aml = H2OAutoML(max_models=20, max_runtime_secs=180, seed=42)
aml.train(x=X, y=y, training_frame=trainf)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_5_AutoML_3_20231102_81555


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    31                 31                          20054                  6            6            6             27            62            46.8064

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.04170561340035965
RMSE: 0.20421952257401751
LogLoss: 0.17947238679498653
Mean Per-Class Error: 0.04559437486266754
AUC: 0.9912326961107448
AUCPR: 0.9886298491998093
Gini: 0.9824653922214897

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3799142695715024
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      327  6    0.018    (6.0/333.0)
1      15   190  0.0732   (15.0/205.0)
Total  342  196  0.039    (21.0/538.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.379914     0.947631  159
max f2                       0.265615     0.955598  179
max f0point5                 0.551118     0.969989  146
max accuracy                 0.379914     0.960967  159
max precision                0.972159     1         0
max recall                   0.0936529    1         298
max specificity              0.972159     1         0
max absolute_mcc             0.379914     0.917132  159
max min_per_class_accuracy   0.287413     0.95122   174
max mean_per_class_accuracy  0.265615     0.9559    179
max tns                      0.972159     333       0
max fns                      0.972159     204       0
max fps                      0.0272029    333       399
max tps                      0.0936529    205       298
max tnr                      0.972159     1         0
max fnr                      0.972159     0.995122  0
max fpr                      0.0272029    1         399
max tpr                      0.0936529    1         298

Gains/Lift Table: Avg response rate: 38.10 %, avg score: 38.22 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0111524                   0.96691            2.62439    2.62439            1                0.968658   1                           0.968658            0.0292683       0.0292683                  162.439   162.439            0.0292683
2        0.0204461                   0.964809           2.62439    2.62439            1                0.966193   1                           0.967537            0.0243902       0.0536585                  162.439   162.439            0.0536585
3        0.0315985                   0.964427           2.62439    2.62439            1                0.964632   1                           0.966512            0.0292683       0.0829268                  162.439   162.439            0.0829268
4        0.0408922                   0.963322           2.62439    2.62439            1                0.963785   1                           0.965892            0.0243902       0.107317                   162.439   162.439            0.107317
5        0.0501859                   0.960913           2.62439    2.62439            1                0.962162   1                           0.965201        

In [ ]:
best_model = aml.leader

varimp = best_model.varimp()
varimp_df = pd.DataFrame(varimp, columns=["Feature", "Relative Importance", "Scaled Importance", "Percentage"])
varimp_df

,Feature,Relative Importance,Scaled Importance,Percentage
0,Sex,117.131447,1.000000,0.277895
1,Age,67.005905,0.572057,0.158972
2,Ticket,57.927380,0.494550,0.137433
3,Pclass,49.098160,0.419171,0.116486
4,Fare,48.412014,0.413314,0.114858
5,Cabin,34.071083,0.290879,0.080834
6,SibSp,20.814650,0.177703,0.049383
7,Parch,15.681441,0.133879,0.037204
8,Embarked,11.353739,0.096932,0.026937


## Test and Submission

In [ ]:
pred = aml.leader.predict(testframe)
predd = pred.as_data_frame()
sub.to_csv('/content/drive/MyDrive/Colab Notebooks/sub2.csv', index=False, header=True)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
